In [1]:
# Import the required libraries
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# RGB Histogram class
class RGBHistogram:
    def __init__(self, bins):
        self.bins = bins
        
    def describe(self, image, mask=None):
        hist = cv2.calcHist([image], [0, 1, 2], mask, self.bins, [0, 256, 0, 256, 0, 256])
        cv2.normalize(hist, hist)
        return hist.flatten()

The RGBHistogram class is used to encapsulate how the flower images are quantified. The
__init__ method on Line 4 takes only a single argument a list containing the number of bins for the 3D histogram. Describing the image will be handled by the describe method , which takes two parameters, an image that the color histogram will be built from, and an optional mask. If we  upplies a mask, then only pixels associated with the masked region will be used in constructing the histogram. This allows him to describe only the petals of the image, ignoring the rest of the image (i.e., the background, which is irrelevant to the flower itself).

In [42]:
# Creating a custom mask folder for the flowers image
for path in imagePaths:
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(gray, 40, 160)
    maskColor = cv2.applyColorMap(canny, cv2.COLORMAP_SPRING)
    flowerName = str(path).split("/")[-1]
    cv2.imwrite("data/flowersColorMask/"+flowerName, maskColor)

In [43]:
# Path for the masked images we have created
maskPaths = []

for path in os.listdir("data/flowersColorMask"):
    maskPaths.append("data/flowersColorMask/"+path)

In [4]:
# Path for the original images
imagePaths = []

for path in os.listdir("data/flowers"):
    imagePaths.append("data/flowers/"+path)

In [45]:
# List contains the features of the images
data = []
# List that contains the flowers names
target = []

# Object 
desc = RGBHistogram([8, 8, 8])

for (imagePath, maskPath) in zip(imagePaths, maskPaths):
    # Read the image
    image = cv2.imread(imagePath)
    # Read the mask
    mask = cv2.imread(maskPath)
    # Convert the BGR image to gray
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    # Extract the features from the masked image
    features = desc.describe(image, mask)
    
    data.append(features)
    target.append(imagePath.split("/")[-1].split("_")[0])

In [46]:
# Unique labels for flowers
labels = np.unique(target)
# Calling the Label Encoder
le = LabelEncoder()
# Coverting flower names into their corresponding integers
target = le.fit_transform(target)

# Split the data
(X_train, X_test, y_train, y_test) = train_test_split(data, target, test_size=0.3, random_state=42)

# RandomForestClassifier
model = RandomForestClassifier(random_state=84)
model.fit(X_train, y_train)

# Classification Report
print(classification_report(y_test, model.predict(X_test), target_names=labels))

               precision    recall  f1-score   support

bougainvillea       0.78      0.81      0.79        26
      daisies       0.83      0.96      0.89        26
       garden       0.48      0.46      0.47        24
    gardenias       0.81      0.88      0.84        24
     hibiscus       0.89      0.62      0.73        26
   hydrangeas       0.79      0.94      0.86        16
       lilies       0.60      0.79      0.68        19
      orchids       0.73      0.55      0.63        20
      peonies       0.41      0.65      0.50        17
        tulip       0.80      0.36      0.50        22

     accuracy                           0.70       220
    macro avg       0.71      0.70      0.69       220
 weighted avg       0.72      0.70      0.69       220



In [53]:
# Randomly selecting five flower sample for the predictions
for i in np.random.choice(np.arange(0, len(imagePaths)), 5):
    
    imagePath = imagePaths[i]
    maskPath = maskPaths[i]
    
    image = cv2.imread(imagePath)
    mask = cv2.imread(maskPath)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    features = desc.describe(image, mask)
    flower = le.inverse_transform(model.predict([features]))[0]
    name = imagePath.split("/")[-1].split("_")[0]
    print("Actual Image Name: ", name.upper())
    print("Predicted Image Name: ", flower.upper())
    cv2.imshow("Image", image)
    cv2.waitKey(0)

Actual Image Name:  gardenias
Predicted Image Name:  GARDENIAS
Actual Image Name:  tulip
Predicted Image Name:  TULIP
Actual Image Name:  tulip
Predicted Image Name:  TULIP
Actual Image Name:  bougainvillea
Predicted Image Name:  BOUGAINVILLEA
Actual Image Name:  lilies
Predicted Image Name:  LILIES
